In [6]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
from requests.auth import HTTPBasicAuth
import random 
import time
import os
import datetime


In [7]:
consumer_key = 'Lq3V1hAxII2Fv8YSdcrhfA538HzhsfzhelYqSUgzGU0A8AEU'
consumer_secret_key = 'jHJjRSzVQBjCizDYmFg4cqWza2R8TOCcLaecRbXrJQcseLu8Lr0AqYXL1ldkAx8A'

# EPO OAuth2 token URL
token_url = "https://ops.epo.org/3.2/auth/accesstoken"

def get_token():
    # Request access token
    response = requests.post(
        token_url,
        auth=HTTPBasicAuth(consumer_key, consumer_secret_key),
        data={"grant_type": "client_credentials"},
    )
    
    # Extract token
    if response.status_code == 200:
        token = response.json()["access_token"]
        return token
    else:
        print("Failed to retrieve access token.")
        return None

In [8]:
data_to_collect_1 = pd.read_csv('sampled_patents.csv')
data_to_collect_2 = pd.read_csv('patents_foster_1929-1965.csv')
data_to_collect_3 = pd.read_csv('patents_foster_1966-1983.csv')
data_to_collect_4 = pd.read_csv('patents_foster_1984-2023.csv')
data_to_collect_5 = pd.read_csv('patents_foster_2024-2024.csv')

data_to_collect_1 = data_to_collect_1[data_to_collect_1['year'] != 2020]

frame_1 = pd.read_pickle('collected_patents_0-999.pkl')
frame_2 = pd.read_pickle('collected_patents_1000-1899.pkl')
frame_3 = pd.read_pickle('collected_patents_1900-7747.pkl')
frame_4 = pd.read_pickle('collected_patents_7748-9999.pkl')
frame_5 = pd.read_pickle('collected_patents_batch_5.pkl')
frame_6 = pd.read_pickle('collected_patents_batch_6.pkl')
frame_7 = pd.read_pickle('collected_patents_batch_7.pkl')
frame_8 = pd.read_pickle('collected_patents_batch_8.pkl')

doc_numbers_1 = frame_1['doc_number']
doc_numbers_2 = frame_2['doc_number']
doc_numbers_3 = frame_3['doc_number']
doc_numbers_4 = frame_4['doc_number']
doc_numbers_5 = frame_5['doc_number']
doc_numbers_6 = frame_6['doc_number']
doc_numbers_7 = frame_7['doc_number']
doc_numbers_8 = frame_8['doc_number']

#create a df of collected patents to recollecting already colected
all_doc_numbers_collected = pd.concat([
    frame_1['doc_number'],
    frame_2['doc_number'],
    frame_3['doc_number'],
    frame_4['doc_number'],
    frame_5['doc_number'],
    frame_6['doc_number'],
    frame_7['doc_number'],
    frame_8['doc_number']
])


all_patents = pd.concat([
    data_to_collect_1,
    data_to_collect_2, 
    data_to_collect_3,
    data_to_collect_4,
    data_to_collect_5
], ignore_index=True)

# Make sure doc_number is of the same type
all_patents['doc_number'] = all_patents['doc_number'].astype(str)
all_doc_numbers_collected = all_doc_numbers_collected.astype(str)

# Filter out already collected patents
uncollected_patents = all_patents[~all_patents['doc_number'].isin(all_doc_numbers_collected)]

uncollected_patents.reset_index(drop=True, inplace=True)



columns = ['year', 'patent', 'name_of_patenter', 'country', 'doc_number', 'IPCR', 'applicant/inventor']


In [9]:
# Preview
print("Remaining uncollected patents:", len(uncollected_patents))
uncollected_patents.head()

Remaining uncollected patents: 160854


,year,country,doc_number
0,1913,CA,147920
1,1913,US,1050965
2,1913,GB,191228696
3,1913,FR,461376
4,1913,GB,191222003


In [16]:
# Split the DataFrame into two halves
midpoint = len(uncollected_patents) // 2

uncollected_patents_laptop = uncollected_patents.iloc[:midpoint].reset_index(drop=True)
uncollected_patents_desktop = uncollected_patents.iloc[midpoint:].reset_index(drop=True)


In [10]:
def get_patent_info(patent_number, token):
    # Base URL for constituent search
    search_url = "https://ops.epo.org/3.2/rest-services/published-data/search/biblio"

    # Headers with authentication
    headers = {
        "Authorization": f"Bearer {token}",
        "Accept": "application/xml",
        "User-Agent": "ProjectAcademicBot/1.0 (for educational purposes only, contact: collen.ellis@sorbonne-universite.fr)",
        "Connection": "keep-alive"
    }

    # Parameters for the request
    params = {
        "q": f"pn={patent_number}",  # Ensure the format matches the EPO API
        "range": "1"  # Ensure this matches the range in your working URL
    }

    # Make the request
    response = requests.get(search_url, headers=headers, params=params)

    try:
        response = requests.get(search_url, headers=headers, params=params)
        return response.status_code, response.text
    except requests.RequestException as e:
        print(f"[ERROR] Request failed for {patent_number}: {e}")
        return 0, None

In [11]:
def parse_xml(xml_str):
    return ET.fromstring(xml_str)

In [12]:
def extract_patent_name(root):
    ns = {'ep': 'http://www.epo.org/exchange'}
    titles = root.findall('.//ep:invention-title', ns)
    return titles[0].text.strip() if titles else None

In [13]:
def extract_ipcr_categories(root):
    ns = {'ep': 'http://www.epo.org/exchange'}
    categories = root.findall('.//ep:classification-ipcr/ep:text', ns)
    return [cat.text.strip() for cat in categories if cat.text]


In [14]:
def extract_patenter_and_role(root):
    ns = {
        'ops': 'http://ops.epo.org',
        'epo': 'http://www.epo.org/exchange',
        'xlink': 'http://www.w3.org/1999/xlink'
    }

    # Stay on separate tracks: get the container elements
    inventors = root.findall('.//epo:inventors/epo:inventor/epo:inventor-name', ns)
    applicants = root.findall('.//epo:applicants/epo:applicant/epo:applicant-name', ns)

    # Extract text from inside the <name> tag within each parent
    inventor_names = []
    for inv in inventors:
        name_el = inv.find('epo:name', ns)
        if name_el is not None and name_el.text:
            inventor_names.append(name_el.text.strip())

    applicant_names = []
    for app in applicants:
        name_el = app.find('epo:name', ns)
        if name_el is not None and name_el.text:
            applicant_names.append(name_el.text.strip())

    # Return whichever exists
    if inventor_names:
        return inventor_names, 'inventor'
    elif applicant_names:
        return applicant_names, 'applicant'
    else:
        return None, None


In [15]:
collected_df = pd.DataFrame(columns=columns)

token = get_token()

# Iterate over patents and update the DataFrame
for i in range(0, len(uncollected_patents)):
    if os.path.exists("stop.txt"):
            print("Stop file detected. Saving progress and exiting.")
            break
    
    
    now = datetime.datetime.now().time()
    if datetime.time(4, 55) <= now <= datetime.time(5, 35):
        print(f"Skipping API call due to maintenance window at {now.strftime('%H:%M:%S')}")
        time.sleep(60)  # Sleep 1 minute before trying again
        continue  # Skip this iteration without making a request

    if i % 100 == 0:
            token = get_token()

    try:
        row = uncollected_patents.iloc[i]
        country = row['country']
        year = row['year']
        patent_number = row['doc_number']

        # Get status code and content from API
        status_code, content = get_patent_info(patent_number, token)

        if status_code != 200:
            print(f"[WARNING] Status {status_code} for patent {patent_number}. Retrying after 1 minute.")
            time.sleep(60)
            continue

        # Parse the XML response
        root = parse_xml(content)
        patent_title = extract_patent_name(root)
        ipcr_list = extract_ipcr_categories(root)
        patenter_name, role = extract_patenter_and_role(root)

        new_row = {
            'year': year,
            'patent': patent_title,
            'name_of_patenter': patenter_name,
            'country': country,
            'doc_number': patent_number,
            'IPCR': ipcr_list,
            'applicant/inventor': role
        }

        new_row_df = pd.DataFrame([new_row])
        print(new_row)
        collected_df = pd.concat([collected_df, new_row_df], ignore_index=True)

        time.sleep(random.uniform(1, 3))

    except Exception as e:
        print(f"[ERROR] Failed to process patent {patent_number} from {country} ({year}): {e}")
        continue

        




{'year': 1913, 'patent': 'НУТРОМІР ІНДИКАТОРНИЙ ГОРЛОВИННИЙ', 'name_of_patenter': ['Крамаренко Сергій Борисович'], 'country': 'CA', 'doc_number': '147920', 'IPCR': [], 'applicant/inventor': 'inventor'}
{'year': 1913, 'patent': 'En stabil vegetarisk emulsion och en metod att göra denna emulsion', 'name_of_patenter': ['TORNBERG EVA', 'ALFREDSSON EMMA', 'EVA TORNBERG,', 'EMMA ALFREDSSON'], 'country': 'US', 'doc_number': '1050965', 'IPCR': ['A23D   7/   005            A I', 'A23L   2/    02            A I', 'A23L  29/   206            A I'], 'applicant/inventor': 'inventor'}
{'year': 1913, 'patent': 'Improvements relating to Acetylene Gas Generators.', 'name_of_patenter': ['HART JOHN', 'JOHN HART'], 'country': 'GB', 'doc_number': '191228696', 'IPCR': ['C10H   1/    00            A I'], 'applicant/inventor': 'inventor'}
{'year': 1913, 'patent': None, 'name_of_patenter': None, 'country': 'FR', 'doc_number': '461376', 'IPCR': [], 'applicant/inventor': None}
{'year': 1913, 'patent': 'Improveme

In [12]:
#saved_df = pd.concat([collected_df.iloc[:900]], ignore_index=True)
saved_df = pd.concat([collected_df], ignore_index=True)
display(saved_df)



,year,patent,name_of_patenter,country,doc_number,IPCR,applicant/inventor
0,1912,WERKWIJZE VOOR HET REGISTREREN EN ANALYSEREN V...,"[TILLEUIL OLIVIER [BE], DE BRUYNE JORIS [BE], ...",US,1026133,[G06Q 30/ 02 A I],inventor
1,1912,Systeem voor het contactloos opladen van voert...,"[DUMAREY GUIDO [BE], DUMAREY GUIDO]",US,1026243,"[B60L 5/ 00 A I, H02J 50/ ...",applicant
2,1912,DIMINUTION DE L'ACTIVITÉ IMMUNITAIRE PAR MODUL...,"[BARSOTTI ANTHONY MICHAEL [US], CANTLEY ALEXAN...",AT,52897,"[A61K 31/ 222 A I, A61K 31/ 44...",inventor
3,1912,FUSION PROTEINS COMPRISING HIV-1 TAT AND/OR NE...,"[GLAXOSMITHKLINE BIOLOG SA [BE], GLAXOSMITHKLI...",US,1015596,"[A61K 39/ 21 A I, A61K 39/ ...",applicant
4,1912,Electric lift truck with fork leveling,"[JACKOWIAK WOJCIECH [PL], JACKOWIAK, WOJCIECH]",FR,441231,[B66F 9/ 12 A I],inventor
...,...,...,...,...,...,...,...
5160,1919,СПОСІБ ВИГОТОВЛЕННЯ ТЕРМОЧУТЛИВИХ ЕЛЕМЕНТІВ ІЗ...,"[BONDAR VOLODYMYR YOSYPOVYCH [UA], DANILCHENKO...",CH,82640,[],inventor
5161,1919,MEJORAS EN EXTRACCIÓN DE LITIO,"[JARIWALA ANKUR D [NL], SAMS GARY W [NL], JARI...",GB,123331,"[B01D 15/ 18 A I, B01D 15/ ...",inventor
5162,1919,Insole for feet with bunions (Machine-translat...,"[OLIVELLA GARCIA ELENA [ES], OLIVELLA GARCIA, ...",US,1305171,[A61F 5/ 10 A I],inventor
5163,1919,None,None,US,1305195,[],None


In [13]:
saved_df.to_pickle('collected_patents_batch_8.pkl')
